# 다음 블로그 크롤링
#### 마지막 실행일자 : 23.08.17
#### ※ Selenium 3.x 버전에서 실행 및 테스트함

In [ ]:
# 패키지 불러오기
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# 블로그 내용 크롤링 함수
def get_blog_content(blog_url):
    driver.get(blog_url)
    time.sleep(2)
    
    # 네이버 블로그 링크일 때
    if blog_url.startswith('https://blog.naver.com/'):
         # iframe 처리
        iframe = driver.find_element(By.CSS_SELECTOR, "iframe#mainFrame")
        driver.switch_to.frame(iframe)

        # Get the content from the iframe
        iframe_content = driver.page_source
        iframe_soup = BeautifulSoup(iframe_content, "html.parser")
        blog_content = iframe_soup.select_one("div.se-main-container")
    
        # Switch back to the main content
        driver.switch_to.default_content()
        if blog_content:
            return blog_content.get_text("\n", strip=True)  # 본문 가져오기
        return None

    # 다음 블로그 링크일 때
    else: 
        # Get the content
        content = driver.page_source  # 페이지 소스 가져오기
        soup = BeautifulSoup(content, "html.parser")  # BeautifulSoup를 사용하여 페이지 소스 파싱
        
        blog_content = soup.select_one("div.contents_style")
        if blog_content:
            return blog_content.get_text("\n", strip=True)  # 본문 가져오기
        return None

# 크롤링할 키워드 검색한 view탭-블로그
url = "url_input"
response = requests.get(url)

# chromedriver 파일 주소
driver = webdriver.Chrome(executable_path="chromedriver")
driver.get(url)  # 브라우저엪에 url 입력 후 이동

# 페이지가 나올 때까지 대기
try:  
    WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'wrap_badge'))
        )      
except Exception as e :
	print(e)

# 크롤링
if response.status_code==200:  # 서버 응답 체크
    # 빈 리스트 생성
    blog_urls = []
    blog_titles_texts = []
    blog_dates_texts = []


    # 블로그 제목 크롤링
    blog_titles = driver.find_elements(By.CLASS_NAME, 'tit-g.clamp-g') 
    print("수집된 title수:", len(blog_titles))

    for blog_title in blog_titles:  # 블로그 제목에서 텍스트 추출
        title_text = blog_title.text.strip()
        if title_text:
            blog_titles_texts.append(title_text)

    # 게시 날짜 크롤링
    blog_dates = driver.find_elements(By.CLASS_NAME, 'gem-subdesc') 
    
    for blog_date in blog_dates:  # 텍스트 추출
        date_text = blog_date.text.strip()
        if date_text:
            blog_dates_texts.append(date_text)
    print("수집된 date수:", len(blog_dates_texts))

    # 블로그 url 크롤링
    href = soup.select("c-title > strong > a")

    for link in href:
        href = link.get("href")  # "href" 속성의 값
        blog_urls.append(href)
    print("수집된 url수:", len(blog_urls))
    
    # 블로그 내용 크롤링
    blog_contents = []
    for i, blog_url in enumerate(blog_urls):
        time.sleep(2)  
        try:
            content = get_blog_content(blog_url)  
        except:
            content = 'iframe error'
        if content:
            blog_contents.append(content)
            print(i)
        else:
            print(f"블로그 {i+1}의 본문을 크롤링하지 못했습니다.")
            blog_contents.append("크롤링 실패")

    # 결과 출력
    for i in range(len(blog_titles)):
        print(f"블로그 url {i+1}: {blog_urls[i]}") 
        print(f"블로그 날짜 {i+1}: {blog_dates_texts[i]}") 
        print(f"블로그 제목 {i+1}: {blog_titles_texts[i]}")
        print(f"블로그 본문 {i+1}: {blog_contents[i]}")
        print("="*50)
else:
    print("HTTP 요청이 실패하였습니다.")


# 셀레니움 드라이버 종료(창 닫기)
driver.quit()

In [ ]:
# 크롤링된 리스트의 길이 체크
print(len(blog_titles_texts))
print(len(blog_contents))
print(len(blog_urls))
print(len(blog_dates))

In [ ]:
# df로 저장
import pandas as pd
blog_df = pd.DataFrame({"blog_dates": blog_dates_texts, "blog_titles": blog_titles_texts, "blog_contents":blog_contents, "blog_urls":blog_urls})
blog_df

# 전처리

In [ ]:
# 데이터 정제
# 특수문자 삭제
import re
blog_df["blog_titles"] = blog_df["blog_titles"].apply(lambda x: re.sub(r"[^a-zA-Z0-9가-힣\s]", "", x))
blog_df["blog_contents"] = blog_df["blog_contents"].apply(lambda x: re.sub(r"[^a-zA-Z0-9가-힣\s]", "", x))
blog_df.head(5)

In [ ]:
# date 정리
# n시간 전, n분 전 -> 오늘날짜
# n일 전 -> 해당 날짜

import pandas as pd
from datetime import datetime, timedelta
import re

# 오늘 날짜 구하기
today = datetime.now().date()

# 시간 전달을 처리하는 함수 정의
def process_time_ago(time_ago):
    time_ago = time_ago.replace(" ", "")  # 띄어쓰기 제거
    if "시간전" in time_ago:
        hours_ago = int(re.search(r'\d+', time_ago).group())
        return today - timedelta(hours=hours_ago)
    elif "일전" in time_ago:
        days_ago = int(re.search(r'\d+', time_ago).group())
        return today - timedelta(days=days_ago)
    elif "주" in time_ago:
        weeks_ago = int(re.search(r'\d+', time_ago).group())
        return today - timedelta(weeks=weeks_ago)
    elif "분전" in time_ago:
        minutes_ago = int(re.search(r'\d+', time_ago).group())
        return today - timedelta(minutes=minutes_ago)
    else:
        return time_ago

# blog_dates 컬럼의 데이터 처리
blog_df["blog_dates"] = blog_df["blog_dates"].apply(process_time_ago)

# blog_dates 컬럼의 데이터 타입 똑같게
blog_df["blog_dates"] = pd.to_datetime(blog_df["blog_dates"])

# 결과 출력
blog_df.head(10)

In [ ]:
# csv로 저장
blog_df.to_csv("다음_블로그_크롤링.csv", index=False, encoding="utf-8-sig")